# 1.Cargamos los datos

In [112]:
library(caret)
library(dplyr)

In [132]:
datos=read.csv('./Washington_State_HDMA.csv',sep=";",dec=",")

#arreglamos variables que nos causan problemas
datos$Tract <- as.numeric(datos$Tract)
datos$rate_spread <- ifelse(is.na(datos$rate_spread), 1, datos$rate_spread)

#creación de la variable target en binario
datos$Accion_tomada[datos$Accion_tomada == "Aprobado"] <- "Haprobado"


In [133]:
datos = datos %>% 
  filter(!is.na(Ingreso_solicit))

In [134]:
datos= datos[ datos$gravamen!='Not applicable', ]

# 2.Seleccionamos las variables que sean de utilidad

In [136]:
variables=datos[,c(
  "Tract"
  , "rate_spread"
  , "population"                    
  , "minorias"           
  , "nro_viv_ocupadas"
  , "nro_unid_residenciales" 
  , "loan_amount_000s"              
  , "Ingreso_medio_fam"      
  , "Ingreso_solicit"         
  #, "nro_cliente"               
  , "tipo_comprador"           
  , "tipo_propiedad"            
  , "preaprovacion"              
  , "tipo_ocupacion"          
  , "msamd_name"                    
  , "loan_type_name"                
  , "proposito"             
  , "gravamen"              
  #, "hoepa"             
  , "edit_status_name"              
  #, "denegacion2"          
  #, "denegacion1"          
  #, "condado"                   
  , "sexo_coapp"         
  , "Raza_coapp"      
  , "etnia_coapp"   
  #, "census_tract_number"           
  , "Sexo"            
  , "Raza"         
  , "Etnia"      
  , "agency_abbr"
  , "Accion_tomada")]

aprobacion = variables$Accion_tomada=="Haprobado"

# 3.Modelos

## 3.1 Primer modelo con todas las variables

In [137]:
modelo1= glm(aprobacion~Tract+rate_spread+minorias+nro_viv_ocupadas+nro_unid_residenciales+loan_amount_000s+Ingreso_medio_fam+Ingreso_solicit+tipo_comprador+tipo_propiedad+preaprovacion+tipo_ocupacion+loan_type_name+proposito+gravamen+edit_status_name+sexo_coapp+etnia_coapp+Raza+Etnia+agency_abbr, data=variables, family='binomial')
summary(modelo1)

Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"



Call:
glm(formula = aprobacion ~ Tract + rate_spread + minorias + nro_viv_ocupadas + 
    nro_unid_residenciales + loan_amount_000s + Ingreso_medio_fam + 
    Ingreso_solicit + tipo_comprador + tipo_propiedad + preaprovacion + 
    tipo_ocupacion + loan_type_name + proposito + gravamen + 
    edit_status_name + sexo_coapp + etnia_coapp + Raza + Etnia + 
    agency_abbr, family = "binomial", data = variables)

Coefficients: (1 not defined because of singularities)
                                            Estimate Std. Error  z value
(Intercept)                               -1.208e+01  3.167e+02   -0.038
Tract                                      4.722e-03  2.408e-04   19.608
rate_spread                                3.219e+01  1.511e+02    0.213
minorias                                  -2.048e-03  4.271e-04   -4.795
nro_viv_ocupadas                          -9.518e-05  2.523e-05   -3.772
nro_unid_residenciales                     6.875e-05  1.811e-05    3.797
loan_amount_000s    

In [138]:
R21=(modelo1$null.deviance-modelo1$deviance)/modelo1$null.deviance
R21

[1] 0.5960549

### 3.2 Segundo modelo

Hacemos un segundo modelo tomando las variables que nos han salido significativas en el primer modelo.

In [139]:
modelo2 =glm(aprobacion ~ Tract+minorias+nro_viv_ocupadas+nro_unid_residenciales+loan_amount_000s+Ingreso_solicit+tipo_propiedad+preaprovacion+ 
  tipo_ocupacion+loan_type_name+proposito+gravamen+edit_status_name+sexo_coapp+etnia_coapp+Raza+Etnia+agency_abbr, data = variables,family = binomial)

summary(modelo2)


Call:
glm(formula = aprobacion ~ Tract + minorias + nro_viv_ocupadas + 
    nro_unid_residenciales + loan_amount_000s + Ingreso_solicit + 
    tipo_propiedad + preaprovacion + tipo_ocupacion + loan_type_name + 
    proposito + gravamen + edit_status_name + sexo_coapp + etnia_coapp + 
    Raza + Etnia + agency_abbr, family = binomial, data = variables)

Coefficients: (1 not defined because of singularities)
                                            Estimate Std. Error z value
(Intercept)                                1.531e-01  4.384e-02   3.492
Tract                                      1.640e-03  1.597e-04  10.269
minorias                                   4.363e-04  2.703e-04   1.614
nro_viv_ocupadas                           2.685e-04  1.551e-05  17.312
nro_unid_residenciales                    -2.134e-04  1.074e-05 -19.869
loan_amount_000s                          -7.297e-06  8.393e-06  -0.869
Ingreso_solicit                            2.393e-04  3.409e-05   7.019
tipo_propieda

In [140]:
R22=(modelo2$null.deviance-modelo2$deviance)/modelo2$null.deviance
R22

[1] 0.05251689

## 3.3 Tercer modelo

Para este tercer modelo vamos a hacer regresiones logísticas de cada variable seleccionada sobre la target.
Tomaremos las $10$ variables con el p-valor más alto.

In [141]:
pvalores_lista <- list()
for (i in 1:25) {
    nombre <- names(variables)[i]
    modelo=glm(aprobacion~variables[[i]], data=variables, family='binomial')
    pvalor= summary(modelo)$coefficients[,4]
    pvalores_lista[[nombre]] <- pvalor
}

df_resultados <- data.frame(
    Variable = rep(names(pvalores_lista), lengths(pvalores_lista)),
    pValor = unlist(pvalores_lista))

# Ordenar por p-valor (de menor a mayor)
df_resultados <- df_resultados[order(df_resultados$pValor), ]

head(df_resultados[-(1:23),],40)

Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"


,Variable,pValor
,<chr>,<dbl>
Tract.variables[[i]],Tract,2.951901e-218
Ingreso_medio_fam.variables[[i]],Ingreso_medio_fam,7.511379e-187
Etnia.(Intercept),Etnia,2.385990e-186
agency_abbr.variables[[i]]FDIC,agency_abbr,3.705333e-185
etnia_coapp.(Intercept),etnia_coapp,7.926573e-168
"msamd_name.variables[[i]]Seattle, Bellevue, Everett - WA",msamd_name,1.861813e-146
Ingreso_solicit.variables[[i]],Ingreso_solicit,4.648256e-121
Tract.(Intercept),Tract,1.260419e-94
minorias.variables[[i]],minorias,9.537093e-88


In [142]:
modelo3=glm(aprobacion~loan_amount_000s+Tract+Ingreso_medio_fam+Etnia+msamd_name+Ingreso_solicit+etnia_coapp+minorias+Sexo+Etnia, data=datos, family=binomial)
summary(modelo3)


Call:
glm(formula = aprobacion ~ loan_amount_000s + Tract + Ingreso_medio_fam + 
    Etnia + msamd_name + Ingreso_solicit + etnia_coapp + minorias + 
    Sexo + Etnia, family = binomial, data = datos)

Coefficients: (1 not defined because of singularities)
                                                    Estimate Std. Error z value
(Intercept)                                       -3.707e-01  2.163e-01  -1.714
loan_amount_000s                                   1.578e-04  2.366e-05   6.669
Tract                                              2.114e-03  1.462e-04  14.462
Ingreso_medio_fam                                  1.019e-05  3.909e-06   2.608
Etniano info dada                                 -2.701e-02  2.334e-02  -1.157
EtniaNot applicable                                7.599e-02  2.398e-01   0.317
EtniaNot Hispanic or Latino                        1.386e-01  1.677e-02   8.265
msamd_nameBellingham - WA                          1.232e-01  6.526e-02   1.888
msamd_nameBremerton, S

In [143]:
R23=(modelo3$null.deviance-modelo3$deviance)/modelo3$null.deviance
R23

[1] 0.01093939

Vamos a comprobar la tasa de falsos positivos de este modelo para ver si es un modelo adecuado o si se podría buscar algo mejor.

In [144]:
set.seed(123)  # Reproducibilidad
indices <- createDataPartition(y = datos$Accion_tomada, p = 0.7, list = FALSE)

train <- datos[indices, ]
test <- datos[-indices, ]

#esta particion nos servira para todos los modelos en los que veamos la tasa de falsos positivos

In [145]:
#eliminar Outliers
Q1 <- quantile(train[["loan_amount_000s"]], 0.25, na.rm = TRUE)
Q3 <- quantile(train[["loan_amount_000s"]], 0.75, na.rm = TRUE)
IQR <- Q3 - Q1

limite_inferior <- Q1 - 1.5 * IQR
limite_superior <- Q3 + 1.5 * IQR

# Filtrar outliers en train y test (usando límites de train)
train_limpio <- train %>% 
  filter(.data[["loan_amount_000s"]] >= limite_inferior & 
         .data[["loan_amount_000s"]] <= limite_superior)

test_limpio <- test %>% 
  filter(.data[["loan_amount_000s"]] >= limite_inferior & 
         .data[["loan_amount_000s"]] <= limite_superior)


In [146]:
#prediccion y cálculo de falsos positivos

predicciones_prob <- predict(modelo3, newdata = test_limpio, type = "response")
predicciones_clase <- ifelse(predicciones_prob > 0.62, 1, 0)

# Matriz de confusión
matriz_confusion <- table(
  Real = test_limpio$Accion_tomada,
  Predicho = predicciones_clase
)

# Calcular tasa de falsos positivos
FP <- matriz_confusion[1, 2]
TN <- matriz_confusion[1, 1]
FPR <- FP / (FP + TN)

# Calcular tasa de falsos negativos
FN= matriz_confusion[2, 1]
TP <- matriz_confusion[2, 2]
FNR <- FN / (FN + TP)

# Resultados
cat("Matriz de Confusión:\n")
print(matriz_confusion)
cat("\nTasa de falsos positivos (Error Tipo 1):", round(FPR, 3))
cat("\nTasa de falsos negativos (Error Tipo 2):", round(FNR, 3))

Matriz de Confusión:
           Predicho
Real            0     1
  Denegado  11805 25635
  Haprobado 16453 55811

Tasa de falsos positivos (Error Tipo 1): 0.685
Tasa de falsos negativos (Error Tipo 2): 0.228

## 3.4 Modelo final

Como hemos comprobado, la tasa de falsos positivos del último modelo es muy alta. Por ello, queremos hacer un modelo que mejore esto.
Al ver que tomando las 10 variables con mejores p-valores, el modelo no es acertado, así que queremos buscar otros métodos para coger variables que puedan
ser significativas e importantes en un modelo. 

En este caso hemos decidido combinar lo visto en el EDA realizado en nuestro anterior trabajo y las correlaciones entre variables (tanto con la target, como entre las otras).

In [147]:
variables$accion_binaria<- as.numeric(variables$Accion_tomada == "Haprobado")


In [148]:
# Seleccionar solo variables numéricas
numericas <- variables[, sapply(variables, is.numeric)]

# Matriz de correlaciones con la variable objetivo
correlaciones <- cor(numericas, use = "complete.obs")
print('Correlaciones con Acción tomada')
print(correlaciones[,"accion_binaria"])

#variables_utiles <- names(which(abs(correlaciones[, "accion_binaria"]) > 0.03))
#print(variables_utiles)
#print('Correlaciones en general')
print(correlaciones)

[1] "Correlaciones con Acción tomada"
                 Tract            rate_spread             population 
           0.051099638            0.075570577            0.009042690 
              minorias       nro_viv_ocupadas nro_unid_residenciales 
          -0.032165869            0.021623458           -0.003166202 
      loan_amount_000s      Ingreso_medio_fam        Ingreso_solicit 
           0.026670064            0.047229078            0.036405704 
        accion_binaria 
           1.000000000 
                              Tract  rate_spread   population     minorias
Tract                   1.000000000 -0.023694454  0.046412529 -0.423279564
rate_spread            -0.023694454  1.000000000 -0.007160709 -0.006580208
population              0.046412529 -0.007160709  1.000000000  0.153935145
minorias               -0.423279564 -0.006580208  0.153935145  1.000000000
nro_viv_ocupadas        0.381055075 -0.007338393  0.765665074 -0.234703601
nro_unid_residenciales  0.142542110  0.00258

In [149]:
modelo4 = glm(aprobacion~tipo_propiedad+minorias+Ingreso_solicit*loan_amount_000s+proposito+gravamen+Ingreso_solicit*loan_type_name, 
              data=variables, family = binomial)

# Resumen del modelo
summary(modelo4)


Call:
glm(formula = aprobacion ~ tipo_propiedad + minorias + Ingreso_solicit * 
    loan_amount_000s + proposito + gravamen + Ingreso_solicit * 
    loan_type_name, family = binomial, data = variables)

Coefficients:
                                                   Estimate Std. Error z value
(Intercept)                                       7.741e-01  1.915e-02  40.410
tipo_propiedadManufactured housing               -4.149e-01  2.023e-02 -20.512
minorias                                         -4.281e-03  2.268e-04 -18.872
Ingreso_solicit                                   6.427e-04  4.821e-05  13.332
loan_amount_000s                                  1.495e-04  2.481e-05   6.024
propositoHome purchase                            5.624e-01  1.772e-02  31.741
propositoRefinancing                             -4.136e-01  1.730e-02 -23.915
gravamenNo tiene                                 -4.620e-01  3.286e-02 -14.062
gravamensubord                                   -2.368e-01  2.409e-02 

In [150]:
#prediccion y cálculo de falsos positivos

predicciones_prob <- predict(modelo4, newdata = test_limpio, type = "response")
predicciones_clase <- ifelse(predicciones_prob > 0.62, 1, 0)

# Matriz de confusión
matriz_confusion <- table(
  Real = test_limpio$Accion_tomada,
  Predicho = predicciones_clase
)

# Calcular tasa de falsos positivos
FP <- matriz_confusion[1, 2]
TN <- matriz_confusion[1, 1]
FPR <- FP / (FP + TN)

# Calcular tasa de falsos negativos
FN= matriz_confusion[2, 1]
TP <- matriz_confusion[2, 2]
FNR <- FN / (FN + TP)

# Resultados
cat("Matriz de Confusión:\n")
print(matriz_confusion)
cat("\nTasa de falsos positivos (Error Tipo 1):", round(FPR, 3))
cat("\nTasa de falsos negativos (Error Tipo 2):", round(FNR, 3))
R24=(modelo4$null.deviance-modelo4$deviance)/modelo4$null.deviance
R24

Matriz de Confusión:
           Predicho
Real            0     1
  Denegado  24898 12542
  Haprobado 33098 39166

Tasa de falsos positivos (Error Tipo 1): 0.335
Tasa de falsos negativos (Error Tipo 2): 0.458

[1] 0.03820633